### Importing required libraries

In [26]:
import torch
from torch.utils.data import Dataset
import torchvision #provide access to datasets, models, transforms, utils, etc
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt

In [45]:
sum_labels[sum_labels > 0] = 1
sum_labels

out1, out2 = network(images, numbers)
n = sum_labels[0].eq(1.).sum().item()
_, indices = torch.topk(out2[0], n)
print(out2[0])
out2[0][indices] = 1.
out2[0][out2[0] < 1] = 0.
out2[0], sum_labels[0], torch.all(out2[0].eq(sum_labels[0]))

tensor([0.1990, 0.2419, 0.1729, 0.2139, 0.1723], grad_fn=<SelectBackward0>)


(tensor([1., 1., 0., 1., 0.], grad_fn=<SelectBackward0>),
 tensor([1., 1., 1., 0., 0.]),
 tensor(False))

In [46]:
def get_preds_correct(preds, labels):
    return preds.argmax(dim=1).eq(labels).sum().item()

def get_preds_sum_correct(preds, labels, batch_size):
    count = 0
    labels[labels > 0] = 1.
    for i in range(batch_size):
        n =  labels[i].eq(1.).sum().item()
        if n > 0:
            _, index = torch.topk(preds[i], n)
            preds[i][index] = 1.
    preds[preds<1] = 0.
    for i in range(batch_size):
        if torch.all(labels[i].eq(preds[i])):
            count += 1
    return count

In [47]:
get_preds_correct(out1, labels), get_preds_sum_correct(out2, sum_labels, 32)

(16, 9)

### Testing if everything is working fine

In [48]:
import torch.optim as optim

mnist_with_numbers = MNISTWithNumbers()

train_loader = torch.utils.data.DataLoader(
    mnist_with_numbers,
    batch_size=BATCH_SIZE,
    shuffle=True
)

network = Network()

optimizer = optim.Adam(network.parameters(), lr=0.001)

batch = next(iter(train_loader)) #get batch
images, labels, numbers, sum_labels = batch
pred_labels, pred_sums = network(images, numbers)
loss_image = F.cross_entropy(pred_labels, labels) # calculating loss for image
loss_sum = F.cross_entropy(pred_sums, sum_labels) # calculating loss for sum

loss_image.backward(retain_graph=True) #calculate gradients
loss_sum.backward()
optimizer.step() #update weights

print('loss1:', loss_image.item(), loss_sum.item())
pred_labels, pred_sums = network(images, numbers)
loss_image = F.cross_entropy(pred_labels, labels) 
loss_sum = F.cross_entropy(pred_sums, sum_labels)
print('loss2:',loss_image.item(), loss_sum.item())

loss1: 2.301152229309082 1.583652377128601
loss2: 2.300562858581543 1.5824871063232422


In [49]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Assuming that we are on a CUDA machine, this should print a CUDA device:

print(device)

cuda


In [50]:
network = Network()
optimizer = optim.Adam(network.parameters(), lr=0.001)
network.to(device)

for epoch in range(EPOCH):
    
    total_correct_MNIST = 0
    total_correct_sum = 0
    total_loss_MNIST = 0
    total_loss_sum = 0
    total_size = 0

    for batch in train_loader:
        images, labels, numbers, sum_labels = batch
        images, labels, numbers, sum_labels = images.to(device), labels.to(device), numbers.to(device), sum_labels.to(device)
        pred_images, pred_sums = network(images, numbers)
        loss_image = F.cross_entropy(pred_images, labels) # calculating loss for image
        loss_sum = F.cross_entropy(pred_sums, sum_labels) # calculating loss for sum

        optimizer.zero_grad() # set the grads to zero 
        loss_image.backward(retain_graph=True)
        loss_sum.backward()
        optimizer.step() # update weights

        total_loss_MNIST += loss_image.item()
        total_correct_MNIST += get_preds_correct(pred_images, labels)
        total_loss_sum += loss_sum.item()
        total_correct_sum += get_preds_sum_correct(pred_sums, sum_labels, len(sum_labels))
        total_size += len(images)

    print(f'epoch: {epoch} \n \t MNIST: total loss: {total_loss_MNIST: .2f} acc: {total_correct_MNIST/total_size*100: .2f} \n \t Addition: total loss: {total_loss_sum: .2f} acc: {total_correct_sum/total_size*100: .2f}')

epoch: 0 
 	 MNIST: total loss:  841.63 acc:  66.52 
 	 Addition: total loss:  716.23 acc:  30.84
epoch: 1 
 	 MNIST: total loss:  742.74 acc:  87.79 
 	 Addition: total loss:  706.50 acc:  34.83
epoch: 2 
 	 MNIST: total loss:  714.16 acc:  94.09 
 	 Addition: total loss:  666.46 acc:  43.20
epoch: 3 
 	 MNIST: total loss:  693.29 acc:  98.48 
 	 Addition: total loss:  640.79 acc:  49.05
epoch: 4 
 	 MNIST: total loss:  691.48 acc:  98.83 
 	 Addition: total loss:  633.88 acc:  50.19
epoch: 5 
 	 MNIST: total loss:  690.57 acc:  98.96 
 	 Addition: total loss:  631.04 acc:  51.63
epoch: 6 
 	 MNIST: total loss:  689.55 acc:  99.17 
 	 Addition: total loss:  628.63 acc:  54.28
epoch: 7 
 	 MNIST: total loss:  688.89 acc:  99.29 
 	 Addition: total loss:  627.46 acc:  54.55
epoch: 8 
 	 MNIST: total loss:  688.53 acc:  99.37 
 	 Addition: total loss:  626.80 acc:  56.18
epoch: 9 
 	 MNIST: total loss:  688.17 acc:  99.43 
 	 Addition: total loss:  626.05 acc:  57.03
epoch: 10 
 	 MNIST: